In [211]:
import pandas as pd

DATA=pd.read_csv('Audiobooks_data.csv')

In [212]:
DATA

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,4.61,0,8.91,0.00,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14078,28220,1620.0,1620,5.33,5.33,1,9.00,0.61,988.2,0,4,0
14079,28671,1080.0,1080,6.55,6.55,1,6.00,0.29,313.2,0,29,0
14080,31134,2160.0,2160,6.14,6.14,0,8.91,0.00,0.0,0,0,0
14081,32832,1620.0,1620,5.33,5.33,1,8.00,0.38,615.6,0,90,0


# New Section

In [213]:
import numpy as np
from sklearn import preprocessing


In [214]:
raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]

In [215]:
num_one_targets=int(np.sum(targets_all))
print(num_one_targets)

2237


In [216]:
targets_all.shape[0]

14084

In [217]:
zero_targets_counter=0
indices_to_remove=[]

for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
      zero_targets_counter += 1
      if zero_targets_counter > num_one_targets:
         indices_to_remove.append(i)

unscaled_inputs_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)

STANDARDIZE THE INPUTS


In [218]:
scaled_inputs=preprocessing.scale(unscaled_inputs_equal_priors)
scaled_targerts=preprocessing.scale(targets_equal_priors)

SHUFFLE THE DATA

In [219]:
shuffled_indices= np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs= scaled_inputs[shuffled_indices]
shuffled_targets= targets_equal_priors[shuffled_indices]

SPLIT THE DATASET INTO TRAIN,VALIDATION & TEST.

In [220]:
samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.7*samples_count)
validation_samples_count=int(0.2*samples_count)
test_samples_count=samples_count-(train_samples_count+validation_samples_count)

print(train_samples_count,validation_samples_count,test_samples_count)

3131 894 449


In [221]:
train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]

In [222]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

ML ALGORITHMN


In [223]:
npz= np.load('Audiobooks_data_train.npz')

train_inputs , train_targets=npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz= np.load('Audiobooks_data_validation.npz')

validation_inputs , validation_targets=npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz= np.load('Audiobooks_data_test.npz')

test_inputs , test_targets=npz['inputs'].astype(np.float), npz['targets'].astype(np.int)


<ipython-input-223-6e56aa681ef2>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs , train_targets=npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
<ipython-input-223-6e56aa681ef2>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/

In [224]:
import tensorflow as tf


In [225]:
input_size=10
output_size=2
hidden_layer_size=50

model=tf.keras.Sequential(
    [
        
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
        tf.keras.layers.Dense(output_size,activation='softmax')
        


    ]





)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

batch_size=50

max_epochs=100

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)


model.fit(train_inputs,train_targets,batch_size=batch_size,
          epochs=max_epochs,callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)




Epoch 1/100
63/63 - 1s - loss: 0.6092 - accuracy: 0.6337 - val_loss: 0.4708 - val_accuracy: 0.7595 - 559ms/epoch - 9ms/step
Epoch 2/100
63/63 - 0s - loss: 0.4689 - accuracy: 0.7442 - val_loss: 0.3938 - val_accuracy: 0.7942 - 130ms/epoch - 2ms/step
Epoch 3/100
63/63 - 0s - loss: 0.4123 - accuracy: 0.7780 - val_loss: 0.3575 - val_accuracy: 0.8188 - 108ms/epoch - 2ms/step
Epoch 4/100
63/63 - 0s - loss: 0.3862 - accuracy: 0.7889 - val_loss: 0.3434 - val_accuracy: 0.7942 - 127ms/epoch - 2ms/step
Epoch 5/100
63/63 - 0s - loss: 0.3720 - accuracy: 0.7934 - val_loss: 0.3305 - val_accuracy: 0.8177 - 139ms/epoch - 2ms/step
Epoch 6/100
63/63 - 0s - loss: 0.3638 - accuracy: 0.7943 - val_loss: 0.3265 - val_accuracy: 0.8333 - 135ms/epoch - 2ms/step
Epoch 7/100
63/63 - 0s - loss: 0.3600 - accuracy: 0.8026 - val_loss: 0.3234 - val_accuracy: 0.8322 - 142ms/epoch - 2ms/step
Epoch 8/100
63/63 - 0s - loss: 0.3522 - accuracy: 0.8074 - val_loss: 0.3249 - val_accuracy: 0.8266 - 126ms/epoch - 2ms/step
Epoch 9/

In [226]:
test_targets.shape

(449,)

In [227]:
test_loss, test_accuracy=model.evaluate(test_inputs,test_targets)

15/15 [==============================] - 0s 2ms/step - loss: 0.3374 - accuracy: 0.8285
